### COVID-19 data preparation and normalization

* Author: Ramiro Aznar
* Data Source: [Datadista](https://github.com/datadista/datasets/tree/master/COVID%2019)
* Last update: 2020-03-15

In [0]:
!pip install cartoframes

In [0]:
import pandas as pd

### 1 | Get data

In [0]:
url = "https://raw.githubusercontent.com/datadista/datasets/master/COVID%2019/ccaa_covid19_{}.csv"

casos = pd.read_csv(url.format('casos'), index_col=0)
fallecidos = pd.read_csv(url.format('fallecidos'), index_col=0)

In [32]:
casos.head()

,CCAA,27/02/2020,28/02/2020,02/03/2020,03/03/2020,04/03/2020,05/03/2020,06/03/2020,09/03/2020,10/03/2020,11/03/2020,12/03/2020,13/03/2020,14/03/2020,15/03/2020
cod_ine,,,,,,,,,,,,,,,
1,Andalucía,1,6,12,13,13,12,21,54,71,90,115,219,269,437
2,Aragón,0,0,0,0,0,1,6,13,38,45,64,80,80,147
3,Asturias,0,0,1,1,2,5,5,7,22,32,47,67,92,137
4,Baleares,1,1,2,2,5,6,6,11,13,16,22,30,28,28
5,Canarias,6,6,7,7,7,8,11,22,25,37,51,70,90,109


In [69]:
fallecidos.head()

,CCAA,03/03/2020,04/03/2020,05/03/2020,06/03/2020,09/03/2020,10/03/2020,11/03/2020,12/03/2020,13/03/2020,14/03/2020,15/03/2020
cod_ine,,,,,,,,,,,,
1,Andalucía,0,0,0,0,0,0,0,0,0,2,6
2,Aragón,0,0,0,1,1,3,4,6,7,7,7
3,Asturias,0,0,0,0,0,0,0,1,1,1,1
4,Baleares,0,0,0,0,0,0,0,1,1,1,1
5,Canarias,0,0,0,0,0,0,0,0,0,1,1


## 2 | Import data into CARTO

In [47]:
from google.colab import files
uploaded = files.upload()

Saving creds.json to creds.json


In [0]:
from cartoframes.auth import set_default_credentials
from cartoframes import to_carto, read_carto

In [0]:
set_default_credentials('creds.json')

In [0]:
to_carto(fallecidos, 'fallecidos', if_exists='replace')
to_carto(casos, 'casos', if_exists='replace')

## 3 | Unnest date columns and normalize values

In [0]:
query = '''
with data as (
  select 
     d.cartodb_id as original_id,
     to_timestamp(unnest(array['03/03/2020', '04/03/2020', '05/03/2020', '06/03/2020',
       '09/03/2020', '10/03/2020', '11/03/2020', '12/03/2020', '13/03/2020',
       '14/03/2020', '15/03/2020']), 'DD/MM/YYYY') AS date,
     unnest(array[d._03_03_2020, d._04_03_2020, d._05_03_2020, d._06_03_2020,
       d._09_03_2020, d._10_03_2020, d._11_03_2020, d._12_03_2020, d._13_03_2020,
       d._14_03_2020, d._15_03_2020
]) AS value,
     d.ccaa,
     s.the_geom,
     s.population,
     'Datadista' as source,
     'https://github.com/datadista/datasets/tree/master/COVID%2019' as source_url
   from 
    {} d,
    spain_regions s
    where d.ccaa = s.nom_ccaa)
    select *, (value/population) as value_density from data'''

# spain_regions is a dataset with ccaa geometries and population from INE

In [0]:
fallecidos_df = read_carto(query.format('fallecidos'))

In [120]:
fallecidos_df

,original_id,date,value,ccaa,the_geom,population,source,source_url,value_density
0,1,2020-03-03 00:00:00+00,0,Andalucía,"MULTIPOLYGON (((-5.03541 38.72619, -5.03274 38...",8414240.0,Datadista,https://github.com/datadista/datasets/tree/mas...,0.000000
1,1,2020-03-04 00:00:00+00,0,Andalucía,"MULTIPOLYGON (((-5.03541 38.72619, -5.03274 38...",8414240.0,Datadista,https://github.com/datadista/datasets/tree/mas...,0.000000
2,1,2020-03-05 00:00:00+00,0,Andalucía,"MULTIPOLYGON (((-5.03541 38.72619, -5.03274 38...",8414240.0,Datadista,https://github.com/datadista/datasets/tree/mas...,0.000000
3,1,2020-03-06 00:00:00+00,0,Andalucía,"MULTIPOLYGON (((-5.03541 38.72619, -5.03274 38...",8414240.0,Datadista,https://github.com/datadista/datasets/tree/mas...,0.000000
4,1,2020-03-09 00:00:00+00,0,Andalucía,"MULTIPOLYGON (((-5.03541 38.72619, -5.03274 38...",8414240.0,Datadista,https://github.com/datadista/datasets/tree/mas...,0.000000
...,...,...,...,...,...,...,...,...,...
204,19,2020-03-11 00:00:00+00,2,La Rioja,"MULTIPOLYGON (((-2.99723 42.64214, -2.99512 42...",316798.0,Datadista,https://github.com/datadista/datasets/tree/mas...,0.000006
205,19,2020-03-12 00:00:00+00,2,La Rioja,"MULTIPOLYGON (((-2.99723 42.64214, -2.99512 42...",316798.0,Datadista,https://github.com/datadista/datasets/tree/mas...,0.000006
206,19,2020-03-13 00:00:00+00,2,La Rioja,"MULTIPOLYGON (((-2.99723 42.64214, -2.99512 42...",316798.0,Datadista,https://github.com/datadista/datasets/tree/mas...,0.000006
207,19,2020-03-14 00:00:00+00,3,La Rioja,"MULTIPOLYGON (((-2.99723 42.64214, -2.99512 42...",316798.0,Datadista,https://github.com/datadista/datasets/tree/mas...,0.000009


In [0]:
casos_df = read_carto(query.format('casos'))

In [122]:
casos_df

,original_id,date,value,ccaa,the_geom,population,source,source_url,value_density
0,1,2020-03-03 00:00:00+00,13,Andalucía,"MULTIPOLYGON (((-5.03541 38.72619, -5.03274 38...",8414240.0,Datadista,https://github.com/datadista/datasets/tree/mas...,0.000002
1,1,2020-03-04 00:00:00+00,13,Andalucía,"MULTIPOLYGON (((-5.03541 38.72619, -5.03274 38...",8414240.0,Datadista,https://github.com/datadista/datasets/tree/mas...,0.000002
2,1,2020-03-05 00:00:00+00,12,Andalucía,"MULTIPOLYGON (((-5.03541 38.72619, -5.03274 38...",8414240.0,Datadista,https://github.com/datadista/datasets/tree/mas...,0.000001
3,1,2020-03-06 00:00:00+00,21,Andalucía,"MULTIPOLYGON (((-5.03541 38.72619, -5.03274 38...",8414240.0,Datadista,https://github.com/datadista/datasets/tree/mas...,0.000002
4,1,2020-03-09 00:00:00+00,54,Andalucía,"MULTIPOLYGON (((-5.03541 38.72619, -5.03274 38...",8414240.0,Datadista,https://github.com/datadista/datasets/tree/mas...,0.000006
...,...,...,...,...,...,...,...,...,...
204,19,2020-03-11 00:00:00+00,179,La Rioja,"MULTIPOLYGON (((-2.99723 42.64214, -2.99512 42...",316798.0,Datadista,https://github.com/datadista/datasets/tree/mas...,0.000565
205,19,2020-03-12 00:00:00+00,205,La Rioja,"MULTIPOLYGON (((-2.99723 42.64214, -2.99512 42...",316798.0,Datadista,https://github.com/datadista/datasets/tree/mas...,0.000647
206,19,2020-03-13 00:00:00+00,243,La Rioja,"MULTIPOLYGON (((-2.99723 42.64214, -2.99512 42...",316798.0,Datadista,https://github.com/datadista/datasets/tree/mas...,0.000767
207,19,2020-03-14 00:00:00+00,278,La Rioja,"MULTIPOLYGON (((-2.99723 42.64214, -2.99512 42...",316798.0,Datadista,https://github.com/datadista/datasets/tree/mas...,0.000878


In [123]:
to_carto(fallecidos_df, 'fallecidos_df', if_exists='replace', geom_col='the_geom')

Success! Data uploaded to table "fallecidos_df" correctly


In [124]:
to_carto(casos_df, 'casos_df', if_exists='replace', geom_col='the_geom')

Success! Data uploaded to table "casos_df" correctly


## 4 | Get centroids

In [0]:
query_centroids = '''
    select  original_id,
            date,
            value,
            ccaa,
            st_y(st_pointonsurface(the_geom)) as latitude,
            st_x(st_pointonsurface(the_geom)) as longitude,
            st_pointonsurface(the_geom) as the_geom,
            population,
            source,
            source_url,
            value_density
    from {}_df
'''

In [0]:
fallecidos_centroids = read_carto(query_centroids.format('fallecidos'))

In [0]:
to_carto(fallecidos_centroids, 'fallecidos_centroids', if_exists='replace', geom_col='the_geom')

In [0]:
casos_centroids = read_carto(query_centroids.format('casos'))

In [0]:
to_carto(casos_centroids, 'casos_centroids', if_exists='replace', geom_col='the_geom')